In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.stats import norm
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn import preprocessing
from sklearn.impute import SimpleImputer as Imputer
import os
import utils

In [8]:
DATA_PATH = "/Users/mykola/MLHEP/Wiremind/"
raw_features = pd.read_csv(os.path.join(DATA_PATH, "ds_train.csv"))
all_features = utils.features_preparation(raw_features)

In [10]:
scaled_features = utils.features_scale(all_features)
scaled_features

/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,current_price,days_before_departure,od_destination_time_month,od_destination_time_week,od_destination_time_day,od_destination_time_weekday,od_time_travel,od_destination_time_hourmin,od_origin_time_hourmin,direction_bool,demand,of_holiday,unof_holiday,date_numerical
0,0.788732,0.000000,0.0,0.0,0.033333,0.166667,0.901961,0.231013,0.224219,1,1.0,0,0,2
1,0.788732,0.034483,0.0,0.0,0.033333,0.166667,0.901961,0.231013,0.224219,1,1.0,0,0,2
2,0.788732,0.068966,0.0,0.0,0.033333,0.166667,0.901961,0.231013,0.224219,1,0.0,0,0,2
3,0.788732,0.103448,0.0,0.0,0.033333,0.166667,0.901961,0.231013,0.224219,1,1.0,0,0,2
4,0.788732,0.137931,0.0,0.0,0.033333,0.166667,0.901961,0.231013,0.224219,1,0.0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292645,0.000000,0.862069,1.0,1.0,0.900000,0.666667,0.490196,1.000000,1.000000,0,0.0,0,0,362
292646,0.000000,0.896552,1.0,1.0,0.900000,0.666667,0.490196,1.000000,1.000000,0,0.0,0,0,362
292647,0.000000,0.931034,1.0,1.0,0.900000,0.666667,0.490196,1.000000,1.000000,0,0.0,0,0,362
292648,0.000000,0.965517,1.0,1.0,0.900000,0.666667,0.490196,1.000000,1.000000,0,0.0,0,0,362


In [47]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['l2', 'auc'],
    'learning_rate': 0.003,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 10,
    "num_leaves": 800,  
    "max_bin": 1024,
    "num_iterations": 100000,
    "n_estimators": 1000
}

In [48]:
train_part, validation = train_test_split(scaled_features, test_size=0.2, shuffle=True, random_state=2334)
print(train_part.shape)
print(validation.shape)

(234120, 14)
(58530, 14)


In [49]:
gbm = lgb.LGBMRegressor(**hyper_params)




In [50]:
gbm.fit(train_part[utils.input_features], train_part.demand,
        eval_set=[(validation[utils.input_features], validation.demand)],
        eval_metric='l1',
        early_stopping_rounds=3000)

/usr/local/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002579 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[52]	valid_0's l1: 1.71163	valid_0's l2: 10.639	valid_0's auc: 0.824153
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[53]	valid_0's l1: 1.70831	valid_0's l2: 10.5946	valid_0's auc: 0.824267
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[54]	valid_0's l1: 1.705	valid_0's l2: 10.5505	valid_0's auc: 0.824364
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[55]	valid_0's l1: 1.70188	valid_0's l2: 10.5093	valid_0's auc: 0.82458
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[56]	valid_0's l1: 1.6986	valid_0's l2: 10.4658	valid_0's auc: 0.824662
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[57]	valid_0's l1: 1.69549	valid_0's l2: 10.4251	valid_0's auc: 0.824827
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[58]	valid_0's l1: 1.69223	valid_0's l2: 1

[107]	valid_0's l1: 1.5627	valid_0's l2: 8.78528	valid_0's auc: 0.822388
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[108]	valid_0's l1: 1.55993	valid_0's l2: 8.7526	valid_0's auc: 0.822468
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[109]	valid_0's l1: 1.55721	valid_0's l2: 8.72032	valid_0's auc: 0.822511
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[110]	valid_0's l1: 1.55445	valid_0's l2: 8.6881	valid_0's auc: 0.82259
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[111]	valid_0's l1: 1.55171	valid_0's l2: 8.65536	valid_0's auc: 0.822698
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[112]	valid_0's l1: 1.54901	valid_0's l2: 8.62296	valid_0's auc: 0.822732
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[113]	valid_0's l1: 1.54628	valid_0's l2: 8.5906	valid_0's auc: 0.82284
[LightGBM] [Warning] No further splits

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[164]	valid_0's l1: 1.43319	valid_0's l2: 7.28779	valid_0's auc: 0.824281
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[165]	valid_0's l1: 1.43108	valid_0's l2: 7.26389	valid_0's auc: 0.824336
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[166]	valid_0's l1: 1.429	valid_0's l2: 7.24031	valid_0's auc: 0.824396
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[167]	valid_0's l1: 1.42692	valid_0's l2: 7.21672	valid_0's auc: 0.824449
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[168]	valid_0's l1: 1.42486	valid_0's l2: 7.19328	valid_0's auc: 0.824501
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[169]	valid_0's l1: 1.42279	valid_0's l2: 7.16992	valid_0's auc: 0.824555
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[170]	valid_0's l1: 1.42075	valid

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5232]	valid_0's l1: 0.889693	valid_0's l2: 3.03969	valid_0's auc: 0.835943
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5233]	valid_0's l1: 0.889692	valid_0's l2: 3.03968	valid_0's auc: 0.835943
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5234]	valid_0's l1: 0.889691	valid_0's l2: 3.03967	valid_0's auc: 0.835942
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5235]	valid_0's l1: 0.889688	valid_0's l2: 3.03966	valid_0's auc: 0.835941
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5236]	valid_0's l1: 0.88969	valid_0's l2: 3.03968	valid_0's auc: 0.835941
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5237]	valid_0's l1: 0.88969	valid_0's l2: 3.03969	valid_0's auc: 0.835943
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5238]	valid_0's l1: 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5291]	valid_0's l1: 0.889767	valid_0's l2: 3.04074	valid_0's auc: 0.83588
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5292]	valid_0's l1: 0.889767	valid_0's l2: 3.04076	valid_0's auc: 0.83588
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5293]	valid_0's l1: 0.889767	valid_0's l2: 3.04074	valid_0's auc: 0.835881
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5294]	valid_0's l1: 0.889767	valid_0's l2: 3.04073	valid_0's auc: 0.83588
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5295]	valid_0's l1: 0.889769	valid_0's l2: 3.04071	valid_0's auc: 0.835878
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5296]	valid_0's l1: 0.889771	valid_0's l2: 3.04074	valid_0's auc: 0.835879
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5297]	valid_0's l1: 0

[5352]	valid_0's l1: 0.889893	valid_0's l2: 3.04233	valid_0's auc: 0.835837
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5353]	valid_0's l1: 0.889893	valid_0's l2: 3.04233	valid_0's auc: 0.835836
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5354]	valid_0's l1: 0.889893	valid_0's l2: 3.04233	valid_0's auc: 0.835835
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5355]	valid_0's l1: 0.889893	valid_0's l2: 3.04233	valid_0's auc: 0.835834
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5356]	valid_0's l1: 0.889892	valid_0's l2: 3.04232	valid_0's auc: 0.835833
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5357]	valid_0's l1: 0.889891	valid_0's l2: 3.04229	valid_0's auc: 0.835833
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5358]	valid_0's l1: 0.889889	valid_0's l2: 3.04226	valid_0's auc: 0.835832
[LightGBM] [Warnin

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5410]	valid_0's l1: 0.889962	valid_0's l2: 3.04216	valid_0's auc: 0.835772
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5411]	valid_0's l1: 0.88996	valid_0's l2: 3.04219	valid_0's auc: 0.835771
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5412]	valid_0's l1: 0.889959	valid_0's l2: 3.04223	valid_0's auc: 0.835772
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5413]	valid_0's l1: 0.889956	valid_0's l2: 3.04226	valid_0's auc: 0.835772
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5414]	valid_0's l1: 0.889954	valid_0's l2: 3.04228	valid_0's auc: 0.835771
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5415]	valid_0's l1: 0.889949	valid_0's l2: 3.04229	valid_0's auc: 0.835771
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5416]	valid_0's l1:

LGBMRegressor(bagging_fraction=0.7, bagging_freq=10, feature_fraction=0.9,
              learning_rate=0.003, max_bin=1024, max_depth=10,
              metric=['l2', 'auc'], n_estimators=1000, num_iterations=100000,
              num_leaves=800, objective='regression', task='train', verbose=0)

In [82]:
import joblib

#joblib.dump(gbm, 'lgbm.pkl')
gbm.booster_.save_model('mode.txt')


prediction_validation = np.array(gbm.predict(validation.loc[:, utils.input_features].values),dtype=float)
(prediction_validation.flatten() - np.array(validation.demand)).sum()


400.8430774733811